### Zero-Shot

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)

Let's explore the world of Few-Shot prompting and all it can do!

In [1]:
!pip install openai cohere tiktoken -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


### OpenAI Key

In [19]:
import os
import openai
import getpass

# set the OPENAI_API_KEY environment variable
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [20]:
from openai import OpenAI
client = OpenAI()

### Prompt Helper Functions

These are only meant to display the outputs in a more palatable style - you can call the API directly using the functions as a guide.

Let's focus on extending this a bit, and incorporate a `system` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The system message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the system prompt.

In [25]:
from IPython.display import display, Markdown

def get_response(messages: list) -> str:
  client = OpenAI()
  response = client.chat.completions.create(
      messages=messages,
      model= "gpt-4-1106-preview",#"gpt-3.5-turbo",#,
  )
  del client
  return response.choices[0].message.content

def wrap_prompt(message: str, role: str) -> dict:
    return {"role": role, "content": message}

def m_print(message: str) -> str:
    display(Markdown(message))

Let's see if our model can define nonsense terms for us.

In [26]:
prompt = wrap_prompt("""What is a flimple-dimple?""", "user")

m_print(get_response([prompt]))

As of my last update in April 2023, "flimple-dimple" is not a recognized term in any standard English language dictionaries, scientific terminology, or widely known slang. It appears to be a nonsensical phrase or a made-up word that could have been used in a specific context, such as within a particular community, game, or as part of a private joke.

If "flimple-dimple" has a meaning in a specific context or if it has been coined after my last update, I would not be aware of it. For accurate information, you might want to provide more context or check the most up-to-date sources available.

Let's see if adding a system prompt helps us.

In [27]:
list_of_prompts = [
    wrap_prompt("You are a creative children's author.", "system"),
    wrap_prompt("What is a flimple-dimple?", "user")
]

m_print(get_response(list_of_prompts))

A flimple-dimple is a whimsical creature from the reaches of my creative imagination, ideal for a children's story. Let me introduce you to this delightful being:

In the far-off land of Giggleberry Grove, deep within the whispering Wizzlewood Forest, you might be lucky enough to spot a flimple-dimple if you're very quiet and very still. A flimple-dimple is a small, furry creature, not much larger than a bunny, with a plump, cushiony body that's perfect for cuddling. It has a tuft of shimmering feathers atop its head that change color with its mood: blue for happy, green for curious, and pink for utterly delighted.

The flimple-dimple has big, expressive eyes the color of dewdrop diamonds and a tiny, shiny nose that wiggles when it senses a giggle. Its ears are floppy and soft, perfect for listening to the secrets of the forest or the hushed whispers of children sharing stories.

It’s known for its peculiar feet, which have squishy pads and little tickle-toes that make a “flimple-dimple” sound when it walks – a sound that's almost like a giggle turned into rhythm. That's how this creature got its name!

The flimple-dimple’s diet is just as whimsical as it is, feeding on rainbow sprouts and nectar from the Laughing Lily, the only flower that tickles the flimple-dimple's nose right back as it eats!

Children are enchanted by flimple-dimples because of their gentle nature and their magical ability to sense the innocence and joy in a child’s heart. By night, the flimple-dimple spreads dreams that are spun from candyfloss and little bursts of starlight, ensuring children slumber in a cocoon of happy dreams.

If you ever find yourself in Wizzlewood Forest, remember to carry a pocket full of giggles and a heart full of wonder: you might just befriend a flimple-dimple of your very own.

This is really quite excellent - but isn't necessarily correct.

Let's try modifying our system prompt to see if anything changes.

In [28]:
list_of_prompts[0] = wrap_prompt("You are a creative poet.", "system")

m_print(get_response(list_of_prompts))

In a world where language twists and bends,
A flimple-dimple's not a thing that tends to trends.
Invented in the moment, sprightly and light,
A flimple-dimple dances in fanciful flight.

A flimple could be a flutter, a frill on a fish,
A dimple, a ripple, an impish little wish.
Surrounded by whimsy, it's a creature of glee,
Something so delightful, it giggles with glee.

'Tis a frivolous trifle, a trinket of the mind,
A whimsical riddle that's uniquely designed.
It could be the bounce in a jubilant step,
Or the curve of a smile that's cheekily kept.

In a world of wonder, a flimple-dimple's free,
To be anything you wish, anything you see.
So let's raise our glasses, to the make-believe's simple,
Here's to life's little joys, like the flimple-dimple!

While this is a cool and complete answer - what if we wanted to teach the model what a flimple-dimple is?

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-4-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [31]:
list_of_prompts = [
    wrap_prompt("A 'flimple-dimple' is a magical and wonderful creature. An example of a sentence that uses the word 'flimple-dimple' is:", "user"),
    wrap_prompt("'Look at that marvelous flimple-dimple flying over there!'", "assistant"),
    wrap_prompt("To 'chillain' is to flit about rapidly in excitement. An example of a sentence that uses the words 'flimple-dimple' and 'chillain' is:", "user")
]

m_print(get_response(list_of_prompts))

"The flimple-dimple was chillaining through the mystical forest, its wings buzzing with joy as it darted between the ancient, enchanted trees."

As you can see, leveraging the `assistant` role lets us teach our model new words, despite never needing to train the model.

This is a (simple) example of "In-Context Learning"!

### Learning New Tasks

Now, we're not limited to just teaching our model silly words.

Let's see that in action!

In [30]:
list_of_prompts = [
    wrap_prompt("You provide appropriate sentiments for user's inputs and output them in a JSON object.", "user"),
    wrap_prompt("Man, I hate cheese!", "user"),
    wrap_prompt("{'sentiment' : 'negative'}", "assistant"),
    wrap_prompt("Oh boy, cheese is my favourite!", "user"),
    wrap_prompt("{'sentiment' : 'positive'}", "assistant"),
    wrap_prompt("Cheese is hella mid.", "user"),
    wrap_prompt("{'sentiment' : 'neutral'}", "assistant"),
    wrap_prompt("The Incredible Hulk is the best, and greatest superhero ever!", "user"),
]

m_print(get_response(list_of_prompts))

{"sentiment": "positive"}

Notice how we're able to both instruct our model on the new task - as well as the output format all through prompting!

### Assignment Portion

Use few-shot prompting to allow the model to achieve a task it's not trained to do!